## Canonical HANK using SSJ_Julia

* We illustrate the functionality of our Julia SSJ package by running a simple cannonical HANK model here.

### 1. Defining the model and getting the steady state

* Defining the cannonical HANK model as in Lecture 4/Tutorial 1 of the workshop using our package
* Obtaining ss values and policies while enforcing GE

In [75]:
using PyPlot

In [76]:
"+=
Authors (by last name): Sophia Cho, Ibrahima Diagne, and Anchi (Bryant) Xia
+="

"+=\nAuthors (by last name): Sophia Cho, Ibrahima Diagne, and Anchi (Bryant) Xia\n+="

In [77]:
include("het.jl")
include("graph.jl")

construct (generic function with 1 method)

1.1 Defining the blocks 
* For exogenenous variables, we can simply specifiy the values in the input dictionary.
* This probably will be turned into a "calibrate" block in the future so we do not have to re-type inputs.

In [78]:
# Specify ins and outs
ins_outs = [
    Dict("ins" => ["r", "G", "B", "Y"], "outs" => ["Z"]), # Fiscal block
    Dict("ins" => ["Z", "r"], "outs" => ["A"]), # HA block
    Dict("ins" => ["A", "B"], "outs" => ["asset_mkt"]), # Asset market clearing
    Dict("ins" => ["Y", "C", "G"], "outs" => ["goods_mkt"]), # Goods market clearing
]

4-element Vector{Dict{String, Vector{String}}}:
 Dict("ins" => ["r", "G", "B", "Y"], "outs" => ["Z"])
 Dict("ins" => ["Z", "r"], "outs" => ["A"])
 Dict("ins" => ["A", "B"], "outs" => ["asset_mkt"])
 Dict("ins" => ["Y", "C", "G"], "outs" => ["goods_mkt"])

In [79]:
# Define the fiscal block
dict_in_fiscal = Dict{Any, Any}(
    "B" => 0.8,
    "r" => 0.03,
    "G" => 0.2,
    "Y" => 1.0
)
dict_out_fiscal = Dict{Any, Any}(
    "Z" => nothing
)

"+=
fiscal_function: inputs -> outputs
=+"

function f_fiscal(block::Block)
    B, r, G, Y = block.ins["B"], block.ins["r"], block.ins["G"], block.ins["Y"]
    T = G + r * B
    Z = Y - T
    block.outs["Z"] = Z
end

fiscal_block = Block(dict_in_fiscal, dict_out_fiscal, f_fiscal, ins_outs, "Fiscal")

Block(Dict{Any, Any}("Y" => 1.0, "B" => 0.8, "r" => 0.03, "G" => 0.2), Dict{Any, Any}("Z" => nothing), f_fiscal, "Fiscal", Any[Dict(("Z", "r") => "∂Z/∂r"), Dict(("Z", "G") => "∂Z/∂G"), Dict(("Z", "B") => "∂Z/∂B"), Dict(("Z", "Y") => "∂Z/∂Y")])

In [80]:
# Define the market clearing block
function f_mkt_clearing(block::Block)
    A, B, Y, C, G = block.ins["A"], block.ins["B"], block.ins["Y"], block.ins["C"], block.ins["G"]
    asset_mkt = A - B
    goods_mkt = Y - C - G
    block.outs["asset_mkt"] = asset_mkt
    block.outs["goods_mkt"] = goods_mkt
end

dict_in_mkt = Dict{Any, Any}(
    "A" => nothing,
    "B" => 0.8,
    "Y" => 1.0,
    "C" => nothing,
    "G" => 0.2
)
dict_out_mkt = Dict{Any, Any}(
    "asset_mkt" => nothing,
    "goods_mkt" => nothing
)
mkt_clearing_block = Block(dict_in_mkt, dict_out_mkt, f_mkt_clearing, ins_outs, "Clearing")

Block(Dict{Any, Any}("Y" => 1.0, "B" => 0.8, "A" => nothing, "C" => nothing, "G" => 0.2), Dict{Any, Any}("goods_mkt" => nothing, "asset_mkt" => nothing), f_mkt_clearing, "Clearing", Any[Dict(("goods_mkt", "Y") => "∂goods_mkt/∂Y"), Dict(("goods_mkt", "C") => "∂goods_mkt/∂C"), Dict(("goods_mkt", "G") => "∂goods_mkt/∂G"), Dict(("asset_mkt", "A") => "∂asset_mkt/∂A"), Dict(("asset_mkt", "B") => "∂asset_mkt/∂B")])

In [81]:
# Household block
hh_block = HH_Block(Dict{Any, Any}("Z" => nothing, "r" => nothing), Dict{Any, Any}("A" => nothing, "C" => nothing), "HA", []) # Defining a placeholder for the heterogeneous agent block

HH_Block(Dict{Any, Any}("Z" => nothing, "r" => nothing), Dict{Any, Any}("A" => nothing, "C" => nothing), "HA", Any[], Dict{Any, Any}())

In [82]:
c_hank = DAG_Rep([fiscal_block, hh_block, mkt_clearing_block])

DAG_Rep(Any[Block(Dict{Any, Any}("Y" => 1.0, "B" => 0.8, "r" => 0.03, "G" => 0.2), Dict{Any, Any}("Z" => nothing), f_fiscal, "Fiscal", Any[Dict(("Z", "r") => "∂Z/∂r"), Dict(("Z", "G") => "∂Z/∂G"), Dict(("Z", "B") => "∂Z/∂B"), Dict(("Z", "Y") => "∂Z/∂Y")]), HH_Block(Dict{Any, Any}("Z" => nothing, "r" => nothing), Dict{Any, Any}("A" => nothing, "C" => nothing), "HA", Any[], Dict{Any, Any}()), Block(Dict{Any, Any}("Y" => 1.0, "B" => 0.8, "A" => nothing, "C" => nothing, "G" => 0.2), Dict{Any, Any}("goods_mkt" => nothing, "asset_mkt" => nothing), f_mkt_clearing, "Clearing", Any[Dict(("goods_mkt", "Y") => "∂goods_mkt/∂Y"), Dict(("goods_mkt", "C") => "∂goods_mkt/∂C"), Dict(("goods_mkt", "G") => "∂goods_mkt/∂G"), Dict(("asset_mkt", "A") => "∂asset_mkt/∂A"), Dict(("asset_mkt", "B") => "∂asset_mkt/∂B")])], Dict{Any, Any}("Y" => Set([3, 1]), "B" => Set([3, 1]), "Z" => Set([2]), "A" => Set([3]), "C" => Set([3]), "r" => Set([2, 1]), "G" => Set([3, 1])), Dict{Any, Any}(2 => Set(Any[3]), 3 => Set{Any

* The above cell created the DAG representation of the cannonical HANK model.

In [83]:
calibration = Dict("eis" => 0.5,  # Elasticity of intertemporal substitution
                   "rho_e" => 0.9,  # Persistence of idiosyncratic productivity shocks
                   "sd_e" => 0.92,  # Standard deviation of idiosyncratic productivity shocks
                   "G" => 0.2,  # Government spending
                   "B" => 0.8,  # Government debt
                   "Y" => 1.,  # Output
                   "min_a" => 0.,  # Minimum asset level on the grid
                   "max_a" => 1_000,  # Maximum asset level on the grid
                   "n_a" => 200,  # Number of asset grid points
                   "n_e" => 10,  # Number of productivity grid points
                   "r" => 0.03,
                   "beta" => 0.85)
                   
inputs_to_hh = Dict("eis" => calibration["eis"], 
                    "rho_e" => calibration["rho_e"],
                    "sd_e" => calibration["sd_e"],
                    "n_e" => calibration["n_e"],
                    "min_a" => calibration["min_a"],
                    "max_a" => calibration["max_a"],
                    "n_a" => calibration["n_a"],
                    "r" => calibration["r"],
                    "beta" => calibration["beta"],
                    "eis" => calibration["eis"],
                    "B" => calibration["B"])

Dict{String, Real} with 10 entries:
  "B"     => 0.8
  "max_a" => 1000
  "n_e"   => 10
  "min_a" => 0.0
  "eis"   => 0.5
  "rho_e" => 0.9
  "sd_e"  => 0.92
  "r"     => 0.03
  "n_a"   => 200
  "beta"  => 0.85

In [84]:
### Print steady state exogenous values from DAG
ss_vals, hh_ss_all_vals = DAG_get_ss(c_hank, "None", inputs_to_hh)
println(ss_vals)

Dict{Any, Any}("Z" => 0.776, "A" => 0.7999999999999997, "C" => 0.8000000004849723, "goods_mkt" => -4.849723400646155e-10, "asset_mkt" => -3.3306690738754696e-16)


### 2. Impulse response while enforcing GE

Example 1

* Shock to $G$ and $B$ (deficit financed)
* $r$ adjusts for market clearing
* In short, monetary policy counters fiscal policy to keep output constant.

In [85]:
end_T = 300 # Truncation period

300

In [86]:
# Fiscal shock with deficit finance

rho_G = 0.8 # Shocking Z is as if we are subtracting from Z directly (equivalent to G increase; need to account for increase elsewhere)
dG = 0.01 * rho_G .^ collect(0:end_T-1)
Gs = fill(0.2, end_T) + dG

rho_B = 0.9
dB = cumsum(dG) .* (rho_B .^ collect(0:end_T-1))
Bs_prev = fill(0.8, end_T)
Bs_prev[2:end] += dB[1:end-1]
Bs = fill(0.8, end_T) + dB

rs_ss = fill(0.03, end_T)

taxes = (1 .+ rs_ss) .* Bs_prev .- Bs .+ Gs
plot(100 .* (Gs .- 0.2))
xlabel("Period")
ylabel("% of G above steady state")

PyObject Text(0, 0.5, '% of G above steady state')

The cells below are outdated. The first of which calculated the Jacobian using the direct perturbation method; the second uses the Jacobian to update the guesses using Newton's algorithm. 

In [87]:
# J_r = zeros(end_T, end_T)
# h = 1e-4 # small shock to r 
# Ys_ss = fill(1.0, end_T)
# no_shock = impulse_map(Ys_ss, rs_ss, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)[1]
# for tshock in 1:end_T
#     r_shock = fill(hh_ss_all_vals["r"], 300)
#     r_shock[tshock] = hh_ss_all_vals["r"] + h
#     J_r[:, tshock] = (impulse_map(Ys_ss, r_shock, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)[1] - no_shock) ./ h
#     print(tshock, " ")
# end

In [88]:
# rs = fill(hh_ss_all_vals["r"], end_T)
# Jbar = J_r[1:end-1, 2:end] 
# errs = Float64[]
# for it in 1:100
#     asset_mkt_error, good_mkt_error, impulse = impulse_map(Ys_ss, rs, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)
#     plot(asset_mkt_error, label="iteration $it")
#     err = maximum(abs.(asset_mkt_error[1:end-1])) # solve for asset market clearing at 0, 1, ..., T-2
#     push!(errs, err)
#     if err < 1E-10
#         println("Asset market clearing up to 12 digits after $it iterations")
#         break
#     end
#     rs[2:end] -= Jbar \ asset_mkt_error[1:end-1] # adjust r_1, ..., r_(T-1)
#     println("This is the maximum error $err after iteration $it")
# end
# legend();

In [89]:
include("het.jl")
include("graph.jl")
het_Js = ha_jacobian(hh_ss_all_vals, Dict("r" => Dict("r" => 1), "z" => Dict("z" => 1)), 1.0, 0.8, 0.8, 0.2, end_T, false) # Multiplied by h which is 1e-4
println("het-block Jacobians complete")

het-block Jacobians complete


In [90]:
Ys_ss = fill(1.0, end_T)
no_shock_ss = zeros(end_T, 10)
rs_ss = fill(hh_ss_all_vals["r"], end_T)
println("Intialized steady state sequences")

Intialized steady state sequences


In [91]:
rs = fill(hh_ss_all_vals["r"], end_T)
Jbar = (het_Js["A"]["r"] - 0.8 .* het_Js["A"]["z"])[1:end-1, 2:end] 
errs = Float64[]
for it in 1:100
    asset_mkt_error, good_mkt_error, impulse = impulse_map(Ys_ss, rs, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)
    plot(asset_mkt_error, label="iteration $it")
    err = maximum(abs.(asset_mkt_error[1:end-1])) # Solve for asset market clearing at 0, 1, ..., T-2
    push!(errs, err)
    if err < 1E-10
        println("Asset market clearing up to 12 digits after $it iterations")
        break
    end
    rs[2:end] -= Jbar \ asset_mkt_error[1:end-1] # Adjust r_1, ..., r_(T-1)
    println("This is the maximum error $err after iteration $it")
end
legend();

This is the maximum error 0.024094531908630867 after iteration 1


This is the maximum error 5.843447019937553e-5 after iteration 2


This is the maximum error 9.759626335981153e-7 after iteration 3


This is the maximum error 2.295880774383363e-8 after iteration 4


This is the maximum error 5.538669523019735e-10 after iteration 5


Asset market clearing up to 12 digits after 6 iterations


In [92]:
xlabel("Period")
ylabel("Increase in interest rate (abs)")
plot(100 .* (rs .- 0.03))

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b5296f0>

* Interest rate is 0 (compared to ss) since this is carried over from the previous period (we set interest for the next period).
* General shape makes sense.
* We could have used the other Jacobian for this exercise (clearing the goods market instead of the asset market; same by Walras's law).

In [93]:
rs = fill(0.03, end_T)
Jbar = (-1 .* het_Js["C"]["r"] + 0.8 .* het_Js["C"]["z"])[1:end-1, 2:end] # This is w.r.t. to c; the goods market has it reversed
errs = Float64[]
impulse = nothing
asset_mkt_error = nothing
for it in 1:1000
    asset_mkt_error, goods_mkt_error, impulse = impulse_map(Ys_ss, rs, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)
    plot(goods_mkt_error, label="iteration $it")
    err = maximum(abs.(goods_mkt_error[1:end-1])) # Solve for asset market clearing at 0, 1, ..., T-2
    push!(errs, err)
    if err < 1E-10
        println("Goods market clearing up to 12 digits after $it iterations")
        break
    end
    rs[2:end] -= (Jbar \ goods_mkt_error[1:end-1]) # Adjust r_1, ..., r_(T-1)
    println("This is the maximum error $err after iteration $it")
end
legend();

This is the maximum error 0.008106307959409309 after iteration 1


This is the maximum error 4.36370335988312e-5 after iteration 2


This is the maximum error 9.068211405283133e-6 after iteration 3


This is the maximum error 8.987865762588587e-6 after iteration 4


This is the maximum error 8.017804292714814e-8 after iteration 5


This is the maximum error 1.687581741016686e-10 after iteration 6


Goods market clearing up to 12 digits after 7 iterations


In [94]:
plot(rs .- 0.03)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b7e0580>

Example 2

* We have the same shocks as before.
* We hold $r$ fixed and let $Y$ adjust freely.
* This is the same exercise as the second example in the fiscal policy tutorial.

In [95]:
include("het.jl")
include("graph.jl")
het_Js = ha_jacobian(hh_ss_all_vals, Dict("r" => Dict("r" => 1), "z" => Dict("z" => 1)), 1.0, 0.8, 0.8, 0.2, end_T, false) # Multiplied by h which is 1e-4
println("het-block Jacobians complete")

het-block Jacobians complete


Again, the cells below are outdated, but may be useful for debugging's sake.

In [96]:
# Bs_ss = fill(0.8, end_T)
# Gs_ss = fill(0.2, end_T)
# J_z = zeros(end_T, end_T)
# h = 1e-4 # small shock to r 
# Ys_ss = fill(1.0, end_T)
# no_shock = impulse_map(Ys_ss, rs_ss, Bs_ss, Bs_ss, Gs_ss, hh_ss_all_vals, end_T)[1]
# for tshock in 1:end_T
#     Y_shock = fill(1.0, end_T)
#     Y_shock[tshock] += h
#     J_z[:, tshock] = (impulse_map(Y_shock, rs_ss, Bs_ss, Bs_ss, Gs_ss, hh_ss_all_vals, end_T)[1] - no_shock) ./ h
#     print(tshock, " ")
# end

In [97]:
# maximum(abs.(het_Js["A"]["z"] - J_z))

In [98]:
Ys_update = fill(1.0, end_T)
Jbar = het_Js["A"]["z"][1:end-1, 1:end-1]
errs = Float64[]
impulse = nothing
asset_mkt_error = nothing
for it in 1:1000
    asset_mkt_error, goods_mkt_error, impulse = impulse_map(Ys_update, rs_ss, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)
    plot(asset_mkt_error, label="iteration $it")
    err = maximum(abs.(asset_mkt_error[1:end-1])) # Solve for asset market clearing at 0, 1, ..., T-2
    push!(errs, err)
    if err < 1E-10
        println("Asset market clearing up to 12 digits after $it iterations")
        break
    end
    Ys_update[1:end-1] -= (Jbar \ asset_mkt_error[1:end-1]) # Adjust r_1, ..., r_(T-1)
    println("This is the maximum error $err after iteration $it")
end
legend();

This is the maximum error 0.024094531908630867 after iteration 1


This is the maximum error 7.112085984528616e-5 after iteration 2


This is the maximum error 4.867118731377573e-7 after iteration 3


This is the maximum error 3.2075658795704953e-9 after iteration 4


Asset market clearing up to 12 digits after 5 iterations


In [99]:
plot(Ys_update)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b880b20>

Alternatively, we know the asset market clears, so $dA = dB$. It sufficies to invert one Jacobian.

In [100]:
dz = het_Js["A"]["z"] \ dB
dY = taxes .- 0.224 + dz
plot(dY)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b880dc0>

In [101]:
# Bs_ss = fill(0.8, end_T)
# Gs_ss = fill(0.2, end_T)
# J_cz = zeros(end_T, end_T)
# h = 1e-4 # small shock to r 
# Ys_ss = fill(1.0, end_T)
# no_shock = impulse_map(Ys_ss, rs_ss, Bs_ss, Bs_ss, Gs_ss, hh_ss_all_vals, end_T)[2]
# for tshock in 1:end_T
#     Y_shock = fill(1.0, end_T)
#     Y_shock[tshock] += h
#     J_cz[:, tshock] = (impulse_map(Y_shock, rs_ss, Bs_ss, Bs_ss, Gs_ss, hh_ss_all_vals, end_T)[2] - no_shock) ./ h
#     print(tshock, " ")
# end

In [102]:
# Ys_update = fill(1.0, end_T)
# Jbar = J_cz[1:end-1, 1:end-1]
# errs = Float64[]
# impulse = nothing
# asset_mkt_error = nothing
# for it in 1:1000
#     asset_mkt_error, goods_mkt_error, impulse = impulse_map(Ys_update, rs_ss, Bs, Bs_prev, Gs, hh_ss_all_vals, end_T)
#     plot(goods_mkt_error, label="iteration $it")
#     err = maximum(abs.(goods_mkt_error[1:end-1])) # solve for asset market clearing at 0, 1, ..., T-2
#     push!(errs, err)
#     if err < 1E-10
#         println("Goods market clearing up to 12 digits after $it iterations")
#         break
#     end
#     Ys_update[1:end-1] -= (Jbar \ goods_mkt_error[1:end-1]) # adjust r_1, ..., r_(T-1)
#     println("This is the maximum error $err after iteration $it")
# end
# legend();

* The Jacobian from het_Js["C"]["z"] computes how consumption responds to income shocks, but is not technically correct when updating with the goods market error (does not account for the effect of -$Y$).
* Using the DAG representation and IFT, we can get the IRF directly without needing to run Newton's algorithm.

* Simultaneous shocks to both $B$ and $G$
* We know $dH = \mathcal J^{H,Y}dY - \mathcal J^{H,C}dC - \mathcal J^{H,G}dG$
* $B$ impacts $H$ through changing $C$ indirectly; $G$ impacts $H$ through $C$ indirectly and $G$ directly
* We first solve for the shocks' total effects on $H$
* We then invert to solve for how $Y$ should respond

In [103]:
dZ_dB = zeros(end_T, end_T)
for row in 1:end_T
    dZ_dB[row, row] = 1
    if row > 1
        dZ_dB[row, row - 1] = - 1 - hh_ss_all_vals["r"]
    end
end

H_G = het_Js["C"]["z"] - Matrix(I, end_T, end_T)
H_B = -1 .* het_Js["C"]["z"] * dZ_dB
H_Y = Matrix(I, end_T, end_T) - het_Js["C"]["z"]

d_other = -1 .* (H_G * dG + H_B * dB)
dY_guess = H_Y \ d_other
plot(dY_guess)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b881060>

## 3. Test forward accumulation

In the above examples, we calculated total Jacobians (of some target variable with respect to some shock variable) by hand. Alternatively, given a shock variable (svar) and a target variable (tvar), we could find all the svar - tvar paths: the same path eneters multiplicatively (chain rule), where as different paths are additive. Code-wise, this is implemented recursively using DFS and back-tracking. One neat thing to note is that because we are working with DAGs, there is no need to keep track of visited nodes, since we will never run into a cycle. 

In [104]:
include("het.jl")
include("graph.jl")

construct (generic function with 1 method)

In [105]:
het_Js_no_compound = ha_jacobian(hh_ss_all_vals, Dict("r" => Dict("r" => 1), "z" => Dict("z" => 1)), 1.0, 0.8, 0.8, 0.2, end_T, false) # Multiplied by h which is 1e-4

Dict{String, Dict{Any, Any}} with 2 entries:
  "A" => Dict("r"=>[0.651714 0.148966 … -2.28187e-13 -2.28908e-13; 0.547047 0.7…
  "C" => Dict("r"=>[0.148286 -0.148966 … 2.28179e-13 2.28899e-13; 0.124219 0.18…

We hard code the Jacobians for now; Bence mentioned that it is easy to perform numerical differentiation for the simple blocks.

In [106]:
all_Jacobians = Dict()
all_Jacobians["∂Z/∂B"] = dZ_dB
all_Jacobians["∂Z/∂r"] = -1 * hh_ss_all_vals["A"] .* I
all_Jacobians["∂Z/∂Y"] = I
all_Jacobians["∂Z/∂G"] = -1 .* I
all_Jacobians["∂A/∂Z"] = het_Js_no_compound["A"]["z"]
all_Jacobians["∂A/∂r"] = het_Js_no_compound["A"]["r"]
all_Jacobians["∂C/∂Z"] = het_Js_no_compound["C"]["z"]
all_Jacobians["∂C/∂r"] = het_Js_no_compound["C"]["r"]
all_Jacobians["∂Hg/∂Y"] = I
all_Jacobians["∂Hg/∂C"] = -1 .* I
all_Jacobians["∂Hg/∂G"] = -1 .* I
all_Jacobians["∂Ha/∂A"] = I
all_Jacobians["∂Ha/∂B"] = -1 .* I

UniformScaling{Int64}
-1*I

Solving example 2 using the goods market error:

In [107]:
Y_paths = fw_acc(c_hank, "Y", "goods")
B_paths = fw_acc(c_hank, "B", "goods")
G_paths = fw_acc(c_hank, "G", "goods")

Hg_Y, Hg_Y_sym = construct(Y_paths, all_Jacobians, end_T)
Hg_G, Hg_G_sym = construct(G_paths, all_Jacobians, end_T)
Hg_B, Hg_B_sym = construct(B_paths, all_Jacobians, end_T)

dY_auto = H_Y \ (-1 .* Hg_G * dG + -1 .* Hg_B * dB)
plot(dY_auto)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b881300>

In [108]:
println("Note that construct also returns the symbolic representation of the total Jacobian computed. For instance, The total Jacobian of how the goods market changes with respect to Y is $Hg_Y_sym")

Note that construct also returns the symbolic representation of the total Jacobian computed. For instance, The total Jacobian of how the goods market changes with respect to Y is ∂Hg/∂C * ∂C/∂Z * ∂Z/∂Y + ∂Hg/∂Y


Great, this is the same impulse response as we obtained before when we let $Y$ adjust and hold $r$ fixed. Note we could have also used the asset market error instead. This is done below.

Solving example 2 using the asset market error:

In [109]:
Y_paths = fw_acc(c_hank, "Y", "asset")
B_paths = fw_acc(c_hank, "B", "asset")
G_paths = fw_acc(c_hank, "G", "asset")

Ha_Y, _ = construct(Y_paths, all_Jacobians, end_T)
Ha_G, _ = construct(G_paths, all_Jacobians, end_T)
Ha_B, _ = construct(B_paths, all_Jacobians, end_T)

dY_auto_alt = Ha_Y \ (-1 .* Ha_G * dG + -1 .* Ha_B * dB)
plot(dY_auto_alt)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b8815a0>

Now, we will let $r$ adjust and hold $Y$ fixed to test the other Jacobians.

Solving example 1 using the asset market error:

In [110]:
Ha_r, _ = construct(fw_acc(c_hank,"r","asset"), all_Jacobians, end_T)
Ha_G, _ = construct(fw_acc(c_hank,"G","asset"), all_Jacobians, end_T)
Ha_B, _ = construct(fw_acc(c_hank,"B","asset"), all_Jacobians, end_T)
            
r_auto = Ha_r[1:end-1, 2:end] \ ((-1. * Ha_B * dB - Ha_G * dG)[1:end-1])
plot(r_auto)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b881840>

Finally, let us use the goods market error instead of the asset market error for this case.

Solving example 1 using the goods market error:

In [111]:
Hg_r, _ = construct(fw_acc(c_hank,"r","goods"), all_Jacobians, end_T)
Hg_G, _ = construct(fw_acc(c_hank,"G","goods"), all_Jacobians, end_T)
Hg_B, _ = construct(fw_acc(c_hank,"B","goods"), all_Jacobians, end_T)
            
r_auto = Ha_r[1:end-1, 2:end] \ ((-1. * Ha_B * dB - Ha_G * dG)[1:end-1])
plot(r_auto)

1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x11b881ae0>